In [1]:
#必要なライブラリをインストール
pip install mecab-python3
pip install markovify

In [1]:
#必要なライブラリを読み込む
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import MeCab
import markovify

In [2]:
# 異常震度にログイン

# パスワードの指定
PASS = "password" # 実行するときは、passwordを正しいパスワードに置き換える

# セッションを開始
session = requests.session()

# ログイン
login_info = {
    "bbspass_str_submit_data":PASS,
}

# action
url_login = "https://9205.teacup.com/nobody/bbs/?"
res = session.post(url_login, data=login_info)
res.raise_for_status() # エラーならここで例外を発生させる

# とりあえずHTMLを全部表示してみる
#print(res.text)

In [3]:
# ゲーム業界研究所の投稿者がOREOのテキストデータだけをスクレイピング

#テキストデータを入れる空のリストを作っておく
list =[]

# ゲーム業界研究所のURLをまとめる
URLs = ["https://9205.teacup.com/nobody/bbs/t27/1-100",
        "https://9205.teacup.com/nobody/bbs/t27/101-200",
        "https://9205.teacup.com/nobody/bbs/t27/201-300",
        "https://9205.teacup.com/nobody/bbs/t27/301-400",
        "https://9205.teacup.com/nobody/bbs/t27/401-500",
        "https://9205.teacup.com/nobody/bbs/t27/501-600",
        "https://9205.teacup.com/nobody/bbs/t27/601-700",
        "https://9205.teacup.com/nobody/bbs/t27/701-800",
        "https://9205.teacup.com/nobody/bbs/t27/801-900",
        "https://9205.teacup.com/nobody/bbs/t27/901-1000",
        "https://9205.teacup.com/nobody/bbs/t65/1-100",
        "https://9205.teacup.com/nobody/bbs/t65/101-200",
        "https://9205.teacup.com/nobody/bbs/t65/201-300",
        "https://9205.teacup.com/nobody/bbs/t65/301-400",
        "https://9205.teacup.com/nobody/bbs/t65/401-500",
        "https://9205.teacup.com/nobody/bbs/t65/501-508"
       ]

#それぞれのURLにアクセス
for url in URLs:
    url_gamelab = url
    res = session.get(url_gamelab)
    res.raise_for_status()

    # URL先から、BeautifulSoupオブジェクトを作成
    soup = BeautifulSoup(res.content, 'html.parser')

    # find_all()メソッドで記事を1つずつ取得し、調べる
    for kiji in soup.find_all(class_='kiji'):
        span = soup.find_all("span") # 記事内の全てのspanを取得
        author = ""
        for tag in span:
            try:
                a = tag.get("class").pop(0)
                # 投稿者名を取得
                if a in "name": 
                    author = tag.text # tag.stringではダメだった
                    break
            except:
                pass
        # 投稿者がOREOならば、テキストを取得し、リストに格納
        if author == "OREO":
            contents = kiji.find('dd', class_="text").text
            list.append(contents)

#リストをパンダスデータフレームに変換
df = pd.DataFrame(list, columns = ['Contents'])

print(df)

                                              Contents
0    \nみなさんご存知の通りモンハンの最新情報が公開されました\nまた動画撮ろうかと思いましたが...
1    \nここまでSPARKやC組のKUDなどとバトルしました\nそして近々OWLとの対戦が決定\...
2    \nとりあえず試し撮りしたのをXMedia RecodeでＷＭＶ（Windows Media...
3    \nどうやらWinのムービーメーカーはipodの動画の形式（mov）には\n対応していないら...
4                       \nだめだ\n容量がでかすぎる\nどうしよう・・・・・・\n
..                                                 ...
887  \n10月にPart1だけアップして音沙汰なしとなっていた、黎の軌跡実況動画\n年末年始に合...
888  \n異常震度にお集まりのPlayStationファンのみなさんこんにちは、OREOです\n\...
889  \nヴァイオレット・エヴァーガーデンは、感情を持たない一人の少女が愛を知るまでの物語\n\n...
890  \n「軌跡シリーズ」の前半戦10作品をようやくクリアした\n\nこれで、シリーズ後半戦の幕開...
891  \n2つ前の記事では、BF2042で「俺たちの"戦場"が帰ってくる」という表現をさせてもらっ...

[892 rows x 1 columns]


In [3]:
#パンダスデータフレームを保存
df.to_csv('gamelab_contents.csv')

In [5]:
#データフレームを読み込み、不要な記号などを削除
df = pd.read_csv('gamelab_contents.csv')
df = df.replace( '\n', '', regex=True)
df = df.replace( '\u3000', '', regex=True)
df = df.replace( '\xa0', '', regex=True)
df = df.replace( '「', '', regex=True)
df = df.replace( '」', '', regex=True)
#df = df.replace( '（', '', regex=True)
#df = df.replace( '）', '', regex=True)
#df = df.replace( '\(', '', regex=True)
#df = df.replace( '\)', '', regex=True)
df = df.replace( '>', '', regex=True)
df = df.replace( '<', '', regex=True)
df = df.replace( 'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', regex=True)
df = df.replace( '\'', '', regex=True)
text = df['Contents'].to_list()
print(text)

['みなさんご存知の通りモンハンの最新情報が公開されましたまた動画撮ろうかと思いましたが、低画質が怖いので写真を撮ってくっつけて動画にしようかとそこで質問ですが、もし上の方法で動画を作った場合見てくれますか？4人以上いれば作ります', 'ここまでSPARKやC組のKUDなどとバトルしましたそして近々OWLとの対戦が決定メンバーを授業中に考え(←おい！)4体を育てることにうち2体を育成し迎えたSPARK戦ここまで5連敗の相手以下次回', 'とりあえず試し撮りしたのをXMedia RecodeでＷＭＶ（Windows Media Player等で見られる比較的一般的なやつ）にエンコードさっそく見てみると・・・・・・画質がクソおかしい、知恵袋で調べるとどうやらビットレートの問題らしいそれを調整してもう1度やると・・・ＨＤ画質に！後はこれをムービーメーカーで１５ＭＢに圧縮するだけだ！！大きく前進！！！', 'どうやらWinのムービーメーカーはipodの動画の形式（mov）には対応していないらしいそこで、ＭＯＶをＷＭＶに変換するため、XMedia Recodeというフリーソフトを使うことにしました', 'だめだ容量がでかすぎるどうしよう・・・・・・', '明日でいい？今動画編集しようとしてるのだが・・・ムービーメーカーで動画がでないなのでとりあえず編集無しでＵＰしようかととりあえず試し撮りしたやつをＵＰしてちゃんとできるかを確認', 'ポケモンの初週販売本数は263万7285本！！ヤバいｗｗ殿堂入りして夢の跡地いったらいきなりスリープ６３を使ってきたｗｗなんとかこなすもあと２体次はスリーパーそして最後はムシャーナおかしいな、そう思いホワイトフォレストへ向かう途中、ローテーションバトルをやることに全部６５ｗｗｗｗｗｗｗｗｗゼクロムを中心になんとか打ち勝ったが・・・まずは育てないとなのんびり冒険もできないｗ今週は木曜日が休みなので今日ファミ通が発売！ＴＧＳの情報がかなり多かったＰＳＰの新色などなどというわけでまた動画撮ります乞うご期待！！', 'おひさしぶりです今日もポケモン日和ですね！ネタバレになるのであんまり書けませんが・・・今バッジ7個ダイケンキ４０・・・なみのり、リベンジ等ゼブライカ４１・・・ほうでん、ニトロチャージ等影のエースヨーテリーの最終進化３７・・・こうげきの高さ

In [6]:
#データフレームを文字列に変換
t = str(text)
print(t)

['みなさんご存知の通りモンハンの最新情報が公開されましたまた動画撮ろうかと思いましたが、低画質が怖いので写真を撮ってくっつけて動画にしようかとそこで質問ですが、もし上の方法で動画を作った場合見てくれますか？4人以上いれば作ります', 'ここまでSPARKやC組のKUDなどとバトルしましたそして近々OWLとの対戦が決定メンバーを授業中に考え(←おい！)4体を育てることにうち2体を育成し迎えたSPARK戦ここまで5連敗の相手以下次回', 'とりあえず試し撮りしたのをXMedia RecodeでＷＭＶ（Windows Media Player等で見られる比較的一般的なやつ）にエンコードさっそく見てみると・・・・・・画質がクソおかしい、知恵袋で調べるとどうやらビットレートの問題らしいそれを調整してもう1度やると・・・ＨＤ画質に！後はこれをムービーメーカーで１５ＭＢに圧縮するだけだ！！大きく前進！！！', 'どうやらWinのムービーメーカーはipodの動画の形式（mov）には対応していないらしいそこで、ＭＯＶをＷＭＶに変換するため、XMedia Recodeというフリーソフトを使うことにしました', 'だめだ容量がでかすぎるどうしよう・・・・・・', '明日でいい？今動画編集しようとしてるのだが・・・ムービーメーカーで動画がでないなのでとりあえず編集無しでＵＰしようかととりあえず試し撮りしたやつをＵＰしてちゃんとできるかを確認', 'ポケモンの初週販売本数は263万7285本！！ヤバいｗｗ殿堂入りして夢の跡地いったらいきなりスリープ６３を使ってきたｗｗなんとかこなすもあと２体次はスリーパーそして最後はムシャーナおかしいな、そう思いホワイトフォレストへ向かう途中、ローテーションバトルをやることに全部６５ｗｗｗｗｗｗｗｗｗゼクロムを中心になんとか打ち勝ったが・・・まずは育てないとなのんびり冒険もできないｗ今週は木曜日が休みなので今日ファミ通が発売！ＴＧＳの情報がかなり多かったＰＳＰの新色などなどというわけでまた動画撮ります乞うご期待！！', 'おひさしぶりです今日もポケモン日和ですね！ネタバレになるのであんまり書けませんが・・・今バッジ7個ダイケンキ４０・・・なみのり、リベンジ等ゼブライカ４１・・・ほうでん、ニトロチャージ等影のエースヨーテリーの最終進化３７・・・こうげきの高さ

In [7]:
#MeCabにより、文字列を単語に分かち書き
parsed_text = MeCab.Tagger('-Owakati').parse(t)

In [8]:
#マルコフ連鎖のモデルを作成
text_model = markovify.NewlineText(parsed_text, state_size=2, well_formed=False)

#モデルを保存
with open('learned_data.json', 'w') as f:
    f.write(text_model.to_json())

In [9]:
#モデルの読み込み
with open('learned_data.json', 'r') as f:
    text_model = markovify.Text.from_json(f.read())

#文章の生成
sentence = text_model.make_sentence(tries=100)
sentence=sentence.replace(" ","")
print(sentence)

['みなさんのミスも見れたし','現在編集、変換中です今日から新曲祈り～涙の軌道アルバムのデータを引き継げるようなことだけ発売日:2017年公式サイト:,356でも言ってるのだろうかOREO、Switch、スマホを使ってみた感想としては敗れること6分、回復ができるようだ','いや、まあいいでしょなかなか良く出来ているところから)話は出てきているという3本・音楽と、いうのもいいのかもしれないが、むしろ不思議な力で元に改装したりしないと思います。ほかには笑われたぼくなつのサントラみたいな格好でUAVを動かすも間に合わなかったのもしょうがないみたいだ','俺が引くしかないでしょ、リッチーの本国イギリスに前々から話は急展開の連続で放送されない安倍政権の責任は重大だ','まあそうな名前のスレのが個人的には教師・生徒も楽しんでもらいたい1本なのが楽しそうに歌ってた4k解像度は全て仮のものが公開されます投票期限は12月15日タイトルだけ見てなかったのが、そいつの年賀状のコメント欄をかなりオーバーしてすぐの音はhiBかhiCくらいだったので紹介してます問題は次だ二日目、負けたのはグルグの王、ザングルグだが夏休みを、奇をてらわず真面目に描いているらしいので報告しているところから)話は続きH…でも友達が何カ所もある・難易度でも良かったな立ち回りがかなり増えた岩田社長から発言があるのから、行列が出来て、鉄道交通の要衝となったから、スライドショーでは、異常震度も10年書き溜めてきたDeadman氏の代名詞だルールは以下の4つのファイルにまとめれば、砂漠に囲まれたが、録音して難解なフレーズを弾き始まったのかその名に恥ないセットリスト少し変更はあったから、出来れば空や零を最初に一気に吹き飛んだこうしている基本的に一人デュエルしてきてくれてたものでも君は生まれ変わって行けるそしていつか捨ててきた！今後ともゲーム業界研究所10周年を迎えるというかこのスレで紹介したものがあり、2日ぶりかな？','DSiLLからの記事から引用前スレ1"ＤＳが値下げされないことになる食材はプレイヤーの皆様がUMD版のパッチ公開までのゲームで勝負出来るモンスターや渓流で尻尾を振り回す、いままでに受け付けさえして解散いろいろと心境の変化があったら書くかもしれないから、JCだとストーリー立てされていれば見てくれたよね店員そう